fiona & jay's attempt to predict german credit data... don't give us any loans

# STEP 1-3: Create train/test sets

# Step 0-1

In [1]:
########################################
# STEP 0: import libraries
########################################
import pandas as pd
import sklearn.datasets
import sklearn.decomposition
import sklearn.discriminant_analysis
import sklearn.ensemble
import sklearn.linear_model
import sklearn.neural_network
import sklearn.model_selection
import sklearn.naive_bayes
import sklearn.neighbors
import sklearn.preprocessing
import sklearn.random_projection
import sklearn.tree
import sklearn.svm

In [2]:
########################################
# STEP 1: Load the dataset
########################################

# NOTE:
# This code is problem specific.
# All the other code in this file is generic to work with any dataset.
# The "German Credit" dataset is a famous dataset for predicting whether someone will default on a loan.
# It has been used in previous practicum projects for banks.
# It is a publicly accessible dataset that is similar in format/content to their private datasets.
data = sklearn.datasets.fetch_openml(name='GermanCreditData')
df = data.data
target = data.target
print(f"df.shape={df.shape}")
print(f"target.shape={target.shape}")

df.shape=(1000, 21)
target.shape=(1000,)


# Step 2

In [3]:
# ########################################
# # STEP 2: Apply "non-learned" data transformations
# ########################################


# convert non-numeric columns to numeric
le = sklearn.preprocessing.LabelEncoder()
df = df[df.columns[:]].apply(le.fit_transform)
print(f"df.shape={df.shape}")

# # # apply the polynomial feature map
# # poly = sklearn.preprocessing.PolynomialFeatures(1)
# # df = poly.fit_transform(df)
# # print(f"df.shape={df.shape}")

# # apply a random projection - removed b/c this is adding too much noise

# proj = sklearn.random_projection.GaussianRandomProjection(
#     n_components=120,
#     random_state=42,
#     )

# df = proj.fit_transform(df)
# print(f"df.shape={df.shape}")



df.shape=(1000, 21)


# STEP 3: Create train/test sets

In [4]:
########################################
# STEP 3: Create train/test sets
########################################

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# ensure that the ratios sum to 1.0
epsilon = 1e-10
assert(1 - epsilon <= train_ratio + validation_ratio + test_ratio <= 1 + epsilon)

# create train0/test set
x_train0, x_test, y_train0, y_test = sklearn.model_selection.train_test_split(
    df,
    target,
    test_size=test_ratio,
    random_state=0,
    )
print(f"len(x_train0)={len(x_train0)}")
print(f"len(x_test)={len(x_test)}")

# create train/validation set
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(
    x_train0,
    y_train0,
    test_size=validation_ratio/(train_ratio + validation_ratio),
    random_state=0,
    )
print(f"len(x_train)={len(x_train)}")
print(f"len(x_val)={len(x_val)}")

len(x_train0)=900
len(x_test)=100
len(x_train)=750
len(x_val)=150


# STEP 4-6: Create train/test sets

In [5]:
########################################
# STEP 4: Apply "learned" data transformations
########################################

# scale the data
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(x_train)
x_train0 = scaler.transform(x_train0)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)
print(f"x_train0.shape={x_train0.shape}")

# PCA the data
#pca = sklearn.decomposition.PCA(n_components=20)
#pca.fit(x_train0)
#x_train0 = pca.transform(x_train0)
#x_train = pca.transform(x_train)
#x_test = pca.transform(x_test)
#x_val = pca.transform(x_val)
#print(f"x_train0.shape={x_train0.shape}")

x_train0.shape=(900, 21)


In [6]:
########################################
# STEP 5: Train a model
########################################

# NOTE:
# the models below are listed in the order we covered them in class;
# the parameters are listed in the order of the documentation;
# you are responsible for understanding how all specified parameters impact the runtime and/or statistical errors


model = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    max_depth=3, # decision stump
    min_samples_split=4, # simplifying the decision tree does not improve performance, and after a certain point, decreases performance
    min_samples_leaf=5,
    max_features=None,
    max_leaf_nodes=50,
    random_state=42,
    )
model = sklearn.ensemble.AdaBoostClassifier(
    estimator=model,
    learning_rate=0.5,
    n_estimators=17, # optimal number of estimators for performance, lots more cause overfitting or hurt performance
    )

# model = sklearn.linear_model.LogisticRegression(
#     C=1e1,
#     penalty='l2',
#     dual=False,
#     fit_intercept=True,
#     intercept_scaling=True,
#     max_iter=100,
#     tol=1e-3,
#     random_state=42,
#     verbose=1,
#     )

# quality over quantity for this ensemble -> (complex decisiontreeclassifier, simple AdaBoost)
model.fit(x_train, y_train)

# report validation accuracy
validation_accuracy = model.score(x_val, y_val)
print(f"validation_accuracy={validation_accuracy:0.4f}") #this is like Eout
train_accuracy = model.score(x_train, y_train)
print(f"train_accuracy={train_accuracy:0.4f}") # this is like Ein

validation_accuracy=0.6467
train_accuracy=0.7400


In [7]:
########################################
# STEP 6: Evaluate on test set
########################################

# WARNING:
# this code should be run only once;
# after the hyperparameters have been decided based on the validation performance,
# then the False can be changed to True to run this code
if True:
    model.fit(x_train0, y_train0)
    test_accuracy = model.score(x_test, y_test)
    print(f"test_accuracy={test_accuracy}")

test_accuracy=0.73
